# Fisher Linear Discriminant

In this exercise, you will apply Fisher Linear Discriminant as described in Chapter 3.8.2 of Duda et al. on the UCI Abalone dataset. A description of the dataset is given at the page https://archive.ics.uci.edu/ml/datasets/Abalone. The following two methods are provided for your convenience: 


* **`utils.Abalone.__init__(self)`** reads the Abalone data and instantiates three data matrices of size (1528, 7), (1307, 7), and (1342, 7) corresponding to the three classes in the dataset: *male (M)*, *female (F)*, and *infant (I)*.


* **`utils.Abalone.plot(self,w)`** produces a histogram of the data when projected onto a vector `w`, and where each class is shown in a different color.


Sample code that makes use of these two methods is given below. It loads the data, looks at the shape of instantiated matrices, and plots various projections of the data: (1) projection on the first dimension of the data, and (2) projection on a random direction.

In [ ]:
%matplotlib inline
import utils,numpy

# Load the data
abalone = utils.Abalone()

# Print dataset size for each class
print(abalone.M.shape,abalone.F.shape, abalone.I.shape)

# Project data on the first dimension
w1 = numpy.array([1,0,0,0,0,0,0])
abalone.plot(w1,'projection on the first dimension')

# Project data on a random direction
w2 = numpy.random.normal(0,1,[7])
w2 /= (w2**2).sum()**.5
abalone.plot(w2,'projection on a random direction')

## Implementation (10 + 5 + 5 = 20 P)

* **Create a function `w = fisher(X1,X2)` that takes as input the data for two classes and returns the Fisher linear discriminant.**


* **Create a function `objective(X1,X2,w)` that evaluates the objective defined in Equation 96 of Duda et al. for an arbitrary projection vector `w`.**


* **Create a function `z = phi(X)` that returns a quadratic expansion for each data point `x` in the dataset. Such expansion consists of the vector `x` itself, to which we concatenate the vector of all pairwise products between elements of `x`.** In other words, letting $x = (x_1,\dots,x_d)$ denote the $d$-dimensional data point, the quadratic expansion for this data point is a $d \cdot (d+3)/2$ dimensional vector given by $\phi(x) = (x_i)_{1 \leq i \leq d} \cup (x_i x_j)_{1 \leq i \leq j \leq d}$. For example, the quadratic expansion for $d=2$ is $(x_1,x_2,x_1^2,x_2^2,x_1 x_2)$.

In [ ]:
import numpy as np

def fisher(X1,X2):
    ##### Replace by your code

    # Calculate the mean vectors per class
    X1_mean = np.array([numpy.mean(X1, axis=0)])
    X2_mean = np.array([numpy.mean(X1, axis=0)])

    # Calculate the scatter matrices for the SW (Scatter within) and sum the elements up
    scatter_X1 = np.dot((X1 - X1_mean),(X1 - X1_mean).T)
    scatter_X2 = np.dot((X2 - X2_mean),(X2 - X2_mean).T)

    # Calculate the SW by adding the scatters within classes 
    SW = scatter_X1 +  scatter_X1
    
    W =  numpy.linalg.inv(SW).dot((X1_mean - X2_mean))
    
    return (w/numpy.linalg.norm(W))
    
    '''    
    # Compute the Eigenvalues and Eigenvectors of SW^-1 SB
    eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(SW),SB))
    '''
    #####
    
def objective(X1,X2,w):
    ##### Replace by your code
    X1_mean = X1.mean(axis=0)  
    X2_mean = X2.mean(axis=0) 
    data_mean = (X1_mean - X2_mean).reshape((X1.shape[1],1))
    Sb = np.dot(data_mean.T, data_mean)
    
    X1_centered = X1 - u1
    X2_centered = X2 - u2
    scatter_X1 = np.dot(X1_centered.T, X1_centered)
    scatter_X1 = np.dot(X2_centered.T, X2_centered)
    SW = scatter_X1 + scatter_X2
    
    return w.dot(Sb.dot(w.T))/w.dot(Sw.dot(w.T))

    #####
    
def expand(X):
    ##### Replace by your code
    d = X.shape[1]
    size_after_expansion = d*(d+3)/2
    result = numpy.zeros(int(size_after_expansion))
    for x in X:
        datapoint_exp = x
        for i in range(0,x.size):
            datapoint_exp = numpy.concatenate((datapoint_exp, x[i]*x[i:x.size]))
        result = numpy.vstack((result, datapoint_exp))
    return result[1:]
    #####

## Analysis (5 + 5 = 10 P)

* **Print value of the objective function for each discriminated pair of classes (M/F, M/I, F/I), and for several values of `w`:**

  * `w` is the difference between the mean vectors of the two classes.
  * `w` is the Fisher linear discriminant.
  * `w` is the Fisher linear discriminant (after quadratic expansion of the data).


* **For the simple Fisher linear discriminant, plot a histogram of the projected data for each discriminated pair of classes using the function `utils.Abalone.plot()`.**

In [ ]:
##### REPLACE BY YOUR CODE
%matplotlib inline
M = abalone.M
F = abalone.F
I = abalone.I


u_MF = M.mean(axis=0) - F.mean(axis=0)
u_MI = M.mean(axis=0) - I.mean(axis=0)
u_FI = F.mean(axis=0) - I.mean(axis=0)
print("Means Linear {:.5f} {:.5f} {:.5f} ".format(objective(M,F,u_MF/numpy.linalg.norm(u_MF)  )
                                            , objective(M,I,u_MI/numpy.linalg.norm(u_MI))
                                            , objective(F,I,u_FI/numpy.linalg.norm(u_FI))))

print("Fisher Linear {:.5f} {:.5f} {:.5f} ".format(objective(M,F,fisher(abalone.M,abalone.F) )
                                            , objective(M,I,fisher(abalone.M,abalone.I))
                                            , objective(F,I,fisher(abalone.F,abalone.I))))

print("Fisher Quadratic Linear {:.5f} {:.5f} {:.5f} ".format(objective(expand(M),expand(F),fisher(expand(M),expand(F)) )
                                            , objective(expand(M),expand(I),fisher(expand(M),expand(I)))
                                            , objective(expand(F),expand(I),fisher(expand(F),expand(I)))))

abalone.plot(fisher(abalone.M,abalone.F),"Fisher Linear (Male/Female)")
abalone.plot(fisher(abalone.M,abalone.I),"Fisher Linear (Male/Infant)")
abalone.plot(fisher(abalone.F,abalone.I),"Fisher Linear (Female/Infant)")
#####